#### Телеграм-бот-чат:

In [22]:
# !pip install python_telegram_bot --upgrade

In [51]:
import json
import random
import logging
import pickle

from nltk import edit_distance
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from nltk.stem.snowball import SnowballStemmer
from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

RANDOM_STATE = 42
TOKEN = '5147095577:AAGfjFjbgKfU98gCqmAcKis8rWQ-r2xaYQE'

In [24]:
with open('BOT_CONFIG.json', 'r', encoding='utf-8') as file:
    BOT_CONFIG = json.load(file)

In [25]:
def clean_text(text):
    output_text = ''
    for char in text:
        if char in ' абвгдеёжзийклмнопрстуфхцчшщъыьэюяabcdefghijklmnopqrstuvwxyz0123456789':
            output_text = output_text + char
    return output_text

In [26]:
X = [] 
Y = []

stemmer_rus = SnowballStemmer('russian')
for intent in BOT_CONFIG['intents'].keys():
    try:
        for example in BOT_CONFIG['intents'][intent]['examples']:
            X.append(stemmer_rus.stem(clean_text(example)))
            Y.append(clean_text(intent))
    except KeyError:
        print(BOT_CONFIG['intents'][intent])

{'example': ['Твои задачи', 'Что ты можешь', 'Твои функции'], 'response': ['Поговорить с вами!', 'Провести с вами беседу', 'Заболтать вас :)']}


In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [28]:
len(X_train), len(X_test), len(Y_train), len(Y_test)

(1152, 288, 1152, 288)

In [29]:
vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(2, 4))
X_train_vectorized = vectorizer.fit_transform(X_train).toarray()
X_test_vectorized = vectorizer.transform(X_test).toarray()

In [30]:
vectorizer.get_feature_names()[:10]

[' 1', ' 1 ', ' 10', ' 10 ', ' 3', ' 30', ' 30 ', ' 4', ' 4 ', ' 5']

In [31]:
model = []
model.append(LogisticRegression(random_state=RANDOM_STATE, solver='liblinear', 
                                class_weight='balanced', intercept_scaling = 5))
model.append(RandomForestClassifier(random_state=RANDOM_STATE, min_samples_split=0.3, 
                                    max_leaf_nodes=100, n_estimators=150, class_weight='balanced'))

In [32]:
model[0].fit(X_train_vectorized, Y_train)
model[1].fit(X_train_vectorized, Y_train)

RandomForestClassifier(class_weight='balanced', max_leaf_nodes=100,
                       min_samples_split=0.3, n_estimators=150,
                       random_state=42)

In [48]:
# сохранение обученной модели в файл
with open('model.pkl', 'wb') as file:
    pickle.dump(model[0], file)
with open('model.pkl','rb') as file:
    model[0] = pickle.load(file)

In [49]:
model[0].score(X_train_vectorized, Y_train)

0.8524305555555556

In [34]:
model[1].score(X_train_vectorized, Y_train)

0.6310763888888888

In [50]:
model[0].score(X_test_vectorized, Y_test)

0.21180555555555555

In [36]:
model[1].score(X_test_vectorized, Y_test)

0.2048611111111111

In [37]:
model_num = 0 if (model[0].score(X_test_vectorized, Y_test) > model[1].score(X_test_vectorized, Y_test)) else 1
model_num

0

In [44]:
Y_preds = model[1].predict(X_test_vectorized)
print(classification_report(Y_preds, Y_test))

                                          precision    recall  f1-score   support

                             aboutdrinks       0.00      0.00      0.00         0
                                   abuse       0.00      0.00      0.00         0
                            acquaintance       0.00      0.00      0.00         0
                                  action       1.00      1.00      1.00         1
                                activity       0.00      0.00      0.00         0
                                  adress       0.00      0.00      0.00         2
                                     age       1.00      0.25      0.40         4
                                 angryeq       0.00      0.00      0.00         1
                                  answer       0.00      0.00      0.00         0
                                 apology       0.00      0.00      0.00         0
                                approval       0.00      0.00      0.00         0
               

D:\Users\xiaomi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Users\xiaomi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Users\xiaomi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Users\xiaomi\anaconda3\lib\site-packages\sklearn\metrics\

In [38]:
def get_intent(input_text, model_num):
    return model[model_num].predict(vectorizer.transform([input_text]))[0]

In [39]:
def bot(input_text):
    intent = get_intent(input_text, model_num)
    return random.choice(BOT_CONFIG['intents'][intent]['responses'])

In [40]:
print('Для выхода из диалога напишите \'Выход\'')
while True:
    input_text = input()
    if input_text == '':
        print('Вы ничего не ввели.')
    elif clean_text(input_text.lower()) == 'выход': # clean_text('Выход'.lower()):
        break
    else:
        print(bot(input_text))
    

Для выхода из диалога напишите 'Выход'
выход


In [52]:
# Define a few command handlers. These usually take the two arguments update and
# context.
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def help_command(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, context: CallbackContext) -> None:
    """Echo the user message."""
    update.message.reply_text(bot(update.message.text))


def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater(TOKEN)

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


In [ ]:
main()